## Develop procedure for using `climpred` with ESM4 PPE data

In [36]:
import climpred as cp
from processing import *
from information import *
import pandas as pd

In [3]:
variable = 'tos'
frequency = 'monthly'

In [4]:
control = open_control(variable,frequency)

In [92]:
startyear = ppeDict['startyears'][0]
startmonth = 1

In [93]:
ensemble = open_ensemble(variable,
                         frequency,
                         startyear,
                         startmonth,
                         control=control,
                         verbose=True)

Opening member 1
Opening member 2
Opening member 3
Opening member 4
Opening member 5
Opening member 6
Opening member 7
Opening member 8
Opening member 9
Adding control


In [96]:
init = cftime.DatetimeNoLeap(startyear,startmonth,1)

In [106]:
ensemblenow = ensemble.copy().sel(xh=-150,yh=0,method='nearest')
ensemblenow = ensemblenow.rename({'time':'lead'}).assign_coords({'lead':np.arange(1,121,1)})
ensemblenow = ensemblenow.expand_dims({'init':[init]})
ensemblenow['lead'].attrs['units']='months'
ensemblenow = ensemblenow.chunk({'member':-1,'lead':-1})
ds = xr.Dataset()
ds[variable]=ensemblenow[variable]

In [ ]:
ds['tos'].squeeze().plot(hue='member')

In [105]:
ensemblenow

<xarray.Dataset>
Dimensions:    (init: 1, lead: 120, nv: 2, xq: 720, yq: 576, zl: 75, member: 10)
Coordinates:
  * init       (init) object 0123-01-01 00:00:00
    xh         float64 -149.8
    yh         float64 -0.0
  * lead       (lead) int64 1 2 3 4 5 6 7 8 ... 113 114 115 116 117 118 119 120
  * nv         (nv) float64 1.0 2.0
  * xq         (xq) float64 -299.5 -299.0 -298.5 -298.0 ... 58.5 59.0 59.5 60.0
  * yq         (yq) float64 -77.82 -77.63 -77.45 -77.26 ... 89.58 89.79 90.0
  * zl         (zl) float64 1.0 3.0 5.0 7.0 ... 3.338e+03 3.591e+03 5.111e+03
  * member     (member) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    tos        (init, member, lead) float32 dask.array<chunksize=(1, 10, 120), meta=np.ndarray>
    time_bnds  (init, member, lead, nv) object dask.array<chunksize=(1, 10, 120, 2), meta=np.ndarray>
Attributes:
    filename:          01010101.ocean_month.nc
    title:             ESM4_piControl_D
    associated_files:  areacello: 01010101.ocean_static.nc
    grid_type:         regular
    grid_tile:         N/A
    history:           2022-03-14 17:08:51 -0400: PostProcessing::PPNetCDF::t...

In [103]:
pm = cp.PerfectModelEnsemble(ensemblenow)

In [104]:
pm.plot()

DimensionError: <class 'function'>.plot() does not allow dimensions other than ['init', 'member', 'lead', 'time'], found {'zl', 'xq', 'nv', 'yq'}. Please use .mean({'zl', 'xq', 'nv', 'yq'}) or .isel() before plot.

In [53]:
import cftime

In [81]:
init = np.array([cftime.DatetimeNoLeap(100,1,1)])
lead = np.arange(0,11,1)
member = np.arange(0,5,1)
data = np.random.normal(size=(len(lead),len(member),len(init)))
ds = xr.Dataset()
ds['data'] = xr.DataArray(data,
                  dims=['lead','member','init'],
                  coords={'lead':lead,'member':member,'init':init})
ds['lead'].attrs['units']='years'

In [82]:
pm = cp.PerfectModelEnsemble(ds)

In [84]:
init

array([cftime.DatetimeNoLeap(100, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)